# StyleAdv attack

Prerequisites: You need to add the pre-trained [style transfer models](https://drive.google.com/drive/folders/12ImHH2kJKw1Vs3rDUSRytP3DZYcHdsZw?usp=sharing) (by https://github.com/martiansideofthemoon/style-transfer-paraphrase) to your Google Drive.

Add a shortcut to your Google Drive under /My Drive/style_transfer_paraphrase

In [4]:
!git clone https://github.com/yifengd/StyleAttack.git
!pip install transformers >> /dev/null


fatal: destination path 'StyleAttack' already exists and is not an empty directory.


In [5]:
!git -C /content/StyleAttack pull

Already up to date.


In [6]:
# Mount on Google Drive File System
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%cd /content/StyleAttack/experiments

INPUT_PATH = "../data/clean/ag/test_100.tsv"
INPUT_PATH = "/content/drive/MyDrive/AdversarialXAI/Raw\ data/imdb_test_100.tsv"

MODEL_NAME = "textattack/distilbert-base-uncased-imdb"

cmd = " ".join(["python",
              "attack.py",
              "--model_name", MODEL_NAME,
              "--bert_type", "distilbert-base-uncased",
              "--orig_file_path", INPUT_PATH,
              "--model_dir", "/content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large",
              "--output_file_path", "/content/record_100.log"
])

!{cmd}


/content/StyleAttack/experiments
Downloading: 100% 28.0/28.0 [00:00<00:00, 38.0kB/s]
Downloading: 100% 483/483 [00:00<00:00, 801kB/s]
Downloading: 100% 226k/226k [00:00<00:00, 1.96MB/s]
Downloading: 100% 455k/455k [00:00<00:00, 3.15MB/s]
Some weights of the model checkpoint at /content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.bias', 'transformer.extra_embedding_project.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0% 0/100 [00:00<?, ?it/s]/content/St

In [9]:

# !CUDA_VISIBLE_DEVICES=0 python attack.py --model_name textattack/distilbert-base-uncased-ag-news --bert_type distilbert-base-uncased --orig_file_path ../data/clean/ag/test_100.tsv --model_dir /content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large --output_file_path /content/record_100.log


In [10]:
import pandas as pd

# dfs = []

# for label in label_set:
#   out_file = f'/content/record_100_{label}.log'
#   df = pd.read_table(out_file,sep='\t')
#   dfs.append(df)

# df = pd.concat(dfs, ignore_index=True)

# df

In [14]:
out_file = f'/content/record_100.log'
df = pd.read_table(out_file,sep='\t')

print(df.columns)

df = df.rename(columns={" orig_sent ": "original_text", " adv_sent ": "adversarial_text", 'p_val ': 'p_val', ' labels': 'labels'})
print(df.columns)

print(len(df))

df


Index(['p_val ', ' orig_sent ', ' adv_sent ', ' labels'], dtype='object')
Index(['p_val', 'original_text', 'adversarial_text', 'labels'], dtype='object')
93


,p_val,original_text,adversarial_text,labels
0,-1,"1891: Stalwart, morally upright military doct...","1891: Stalwart, morally upright military doct...",1
1,-1,"Robert Cummings, Laraine Day and Jean Muir st...","Robert Cummings, Laraine Day and Jean Muir st...",1
2,-1,"When Precious Bane aired in America, I was st...","When Precious Bane aired in America, I was st...",1
3,1,This film is terribly bad. Kevin Spacey is a ...,"he's really bad, but he's really good at LA C...",0
4,1,OK i have seen Hershall Gordon Lewis movies b...,"I've seen Hershall Gordon Lewis films before,...",0
...,...,...,...,...
88,-1,I will not spoil your surprise by mentioning ...,I will not spoil your surprise by mentioning ...,0
89,1,"A gem from Japan, where so many of the world'...","in the film, which is being made by the best ...",1
90,-1,"From the opening drum hit of ""We're Gonna Gro...","From the opening drum hit of ""We're Gonna Gro...",1
91,-1,"Not since J. Michael Straczynski's Babylon 5,...","Not since J. Michael Straczynski's Babylon 5,...",1


In [15]:
# df.query("p_val == 1")

In [16]:
import os

out_name = "imdb_test_100_styleadv_distilbert.csv"

out_dir = '/content/drive/MyDrive/AdversarialXAI/Adversarial Samples/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

df.to_csv(os.path.join(out_dir, out_name), index=False)